# Installs and Imports

In [3]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install stanza
#!pip install negate==1.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 25.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import stanza
#from negate import Negator

# Load Data and Models

## Tweet Data

In [5]:
file_name = "exp3.csv"
df = pd.read_csv(file_name)

In [6]:
df

,ID,original_ID,tweet,sarcastic,rephrase
0,5002,test_a_921,I really loved the drive into work today in th...,1,NaN
1,1377,sign_3530,i just looooove getting big cases on friday af...,1,i just hate getting big cases on friday aftern...
2,3297,sign_12376,i just love it when people near and dear to me...,1,no one seems to care that i'm sick
3,2404,train_2773,man I really hate seeing two fucking idiots ra...,0,NaN
4,10364,test_a_873,I really hate being stuck at home thanks to th...,0,NaN
5,13608,train_2450,I absolutely hate when players do that dumb ea...,0,NaN
6,4821,sign_3549,i absolutely love how my mom doesn't believe a...,1,i really don't like how my mom doesn't believe...
7,8042,sign_12009,i really love enrollments,1,i really hate enrollments
8,15087,sign_12389,i just love going down my timeline to see one ...,1,i hate the negative posts on my timeline
9,15507,sign_2773,i just love having to get my car towed on a su...,1,i just hate having to get my car towed on a su...


## Sentiment Classifier

In [7]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def classify_sentiment(text) :
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    # [ negative, neutral, positive ]
    return scores

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Constituency Parser

In [8]:
stanza.download('en')
nlp = stanza.Pipeline('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytor

In [9]:
def width(t) :
  wds = str(t).replace(")", "").split(" ")
  filt = [wd for wd in wds if '(' not in wd]
  return len( filt )

def add_spans(t, tks) :

  if t.is_preterminal() :
    tk = tks[0]
    t.span = ( tk.start_char , tk.end_char )
    t.tk_info = tk
  else :
    start, end = 0, 0
    for child in t.children :
      w = width(child)
      end = end + w
      add_spans(child, tks[start:end])
      start = end
    t.span = ( t.children[0].span[0] , t.children[-1].span[-1] )


def parse_tree(t):
  doc = nlp(t)
  tree = doc.sentences[0].constituency
  tks_list = []
  for item in doc.sentences[0].tokens :
      tks_list += item.words
  add_spans(tree, tks_list)
  return tree

In [10]:
def membership_check(tree, label):

  if tree.is_preterminal():
    return(tree.label == label)

  if tree.label == label :
    return True

  children = tree.children

  for el in children:

    if (membership_check(el, label)):
      return True

  return False
  # if there is no VP, return

def extract_VP(row):
  # the input will be tree.children

  if not isinstance(row, list):
    if not (membership_check(row, 'VP')):
      return None
    row = [row]

  for idx, el in enumerate(row):
    if el.label == 'VP':
      spc = None
      if idx > 0 and 'ADV' in row[idx - 1].label:
        spc = row[idx - 1]
      return {"VP" : el, "specifier": spc}

  l = []

  for el in row:
    l.extend(el.children)

  return extract_VP(l)



In [11]:
def parse_VP(tree):

  if tree.is_preterminal():

    if 'VB' in tree.label or 'MD' in tree.label or 'RB' in tree.label:
      return [tree], []
    return [], [tree]

  if not tree.label.startswith('V'):
    return [], [tree]

  vbs_list = []
  consts_list = []
  for child in tree.children:
    vbs, consts = parse_VP(child)
    vbs_list += vbs
    consts_list += consts

  return vbs_list, consts_list

In [12]:
def get_text(full, span) :
  if not span :
    return None
  return full[span[0] : span[1]]

def parse_tweet(tw, display=False) :
  tree = parse_tree(tw)

  if display :
    print(tw)
    print(tree)

  res = extract_VP(tree)
  if res :
    spc = res['specifier']
    vbs, consts = parse_VP(res["VP"])

    if 'be' in [vb.tk_info.lemma.lower() for vb in vbs] :
      if display :
        print("Special case : be")
      n_consts = []
      for c in consts :
        if 'ADJ' in c.label :
          vbs.append(c)
        else :
          n_consts.append(c)
      consts = n_consts

    spc_span = get_text(tw, spc.span) if spc else None
    if display :
      print(f"specifier : { spc_span}")
      print(f"verbs : {get_text(tw,  ( vbs[0].span[0], vbs[-1].span[-1] ) )}")
      print("constituents : ")
    for c in consts :
      if display :
        print( get_text(tw, c.span) )
  try :
    return {"text" : tw, "specifier" : spc.span if spc else None, "constituents" : [c.span for c in consts], "verb" : ( vbs[0].span[0], vbs[-1].span[-1] ) }
  except :
    return {"text" : tw, "specifier" : None, "constituents" : None, "verb" : None}


## Sentence Negator

In [ ]:
negator = Negator()

# Brute-Force Algorithm for Threshold

In [13]:
def sentiment_distance(tweet, verb, noun_phrase):
  """Calculates the sentiment distance (euclidic distance) between the sentiment scores of the V and NP of a given tweet."""

  tweet_sentiment = classify_sentiment(tweet)
  tweet_no_v_sentiment = classify_sentiment(tweet.replace(verb, "")) ##TO DO: change (ask Samba)
  tweet_no_np_sentiment = classify_sentiment(tweet.replace(noun_phrase, "")) ##TO DO: change (ask Samba)
  v_sentiment = tweet_sentiment - tweet_no_v_sentiment
  np_sentiment = tweet_sentiment - tweet_no_np_sentiment

  return np.linalg.norm(v_sentiment - np_sentiment)

def is_sarc(sentiment_dist, threshold):
  """Returns True if the sentiment distance is greater than a given threshold and False otherwise."""

  return sentiment_dist > threshold

In [21]:
import re

def define_threshold(tweets, gold_annotations, threshold_list):
  """
  Creates a list of tuples containing 1) the tweet body, 2) the sarc/non-sarc gold annotation, 3) the sentiment distance between V and NP.
  Then, iterates over a list of thresholds and for each threshold calculates the accuracy between the gold annotations and the predicted values.
  Returns a dictionary of thresholds and accuracies for those thresholds.
  """

  tweet_annotation_sentiment_distance = []
  for tweet, annotation in zip(tweets, gold_annotations):
    """
    res = parse_tweet(tweet)
    if res["verb"] :
      verb_span = (res["specifier"][0], res["verb"][-1]) if res["specifier"] else res["verb"]
      comp_span = ( min([x[0] for x in res["constituents"]]), max([x[0] for x in res["constituents"]])  )
      tweet_annotation_sentiment_distance.append( (tweet, annotation, sentiment_distance(tweet, get_text(tweet, verb_span), get_text(tweet, comp_span) ) ) )
    else :
    """
    tweet_split = tweet.split()
    verb_idx = min([i for i,x in enumerate(tweet_split) if re.match(r"(?i)lo+v|ha+t", x) ])
    try :
      i_idx = [x.lower() for x in tweet_split].index("i")
    except Exception :
      i_idx = min([i for i,x in enumerate(tweet_split) if x.lower().startswith("i'")])
    verb = " ".join(tweet_split[i_idx + 1:verb_idx + 1])
    noun_phrase = " ".join(tweet_split[verb_idx + 1:])
    tweet_annotation_sentiment_distance.append((tweet, annotation, sentiment_distance(tweet, verb, noun_phrase)))

  # scaling of the data
  sentiment_max = max([x[2] for x in tweet_annotation_sentiment_distance])
  print(sentiment_max)
  tweet_annotation_sentiment_distance = [(t,a, sentiment_dist/sentiment_max) for t, a, sentiment_dist in tweet_annotation_sentiment_distance]
  print(tweet_annotation_sentiment_distance)

  threshold_accuracy_dict = {}
  for threshold in threshold_list:
    correct = 0
    for _, annotation, sentiment_dist in tweet_annotation_sentiment_distance:
      if is_sarc(sentiment_dist, threshold) == bool(annotation):
        correct += 1

    threshold_accuracy_dict[threshold] = correct / len(tweet_annotation_sentiment_distance)

  return threshold_accuracy_dict, tweet_annotation_sentiment_distance

In [22]:
gap = 0.001
threshold_list = list(np.arange(0, 1 + gap, gap))

threshold_accuracy_dict, tweet_annotation_sentiment_distance = define_threshold(df["tweet"], df["sarcastic"], threshold_list)
best_threshold = max(threshold_accuracy_dict, key=threshold_accuracy_dict.get)
print("Best threshold:", best_threshold)
print("Accuracy:", threshold_accuracy_dict[best_threshold])

6.992921
[('I really loved the drive into work today in the dark, wet, heavy rain', 1, 0.902059), ('i just looooove getting big cases on friday afternoons', 1, 0.7179631), ("i just love it when people near and dear to me act like they care that i'm really sick with a fever", 1, 0.8793677), ('man I really hate seeing two fucking idiots raising a baby', 0, 0.082117595), ('I really hate being stuck at home thanks to the governments lack of controls on covid in school.', 0, 0.09723044), ('I absolutely hate when players do that dumb early celebration nonsense', 0, 0.08448678), ("i absolutely love how my mom doesn't believe a damn word i say", 1, 1.0), ('i really love enrollments', 1, 0.56133753), ('i just love going down my timeline to see one of my friends post negativety for the last 2 days', 1, 0.8679203), ('i just love having to get my car towed on a sunday when i work the next day', 1, 0.9746228), ('and i just love cardio', 1, 0.4952784), ('i just love how people do the total opposite 

In [28]:
import json

data_as_lists = [[item[0], item[1], float(item[2])] for item in tweet_annotation_sentiment_distance]

# Save to a JSON file
with open('tweet_annotation_sentiment_distance_exp3.json', 'w') as json_file:
    json.dump(data_as_lists, json_file)

In [23]:
df['prediction'] = None
df['correct'] = None
for i in range(df.shape[0]):
  df.at[i, 'prediction'] = int(is_sarc(tweet_annotation_sentiment_distance[i][2], best_threshold))
  df.at[i, 'correct'] = int(df.at[i, 'prediction'] == df.at[i, 'sarcastic'])

In [24]:
from scipy.stats import binomtest

result = binomtest(sum(df['correct']), df.shape[0], np.mean(df['sarcastic']), alternative='greater')

print(f"p-value: {result.pvalue}")
print(f"Test statistic: {result.statistic}")
print(result)

p-value: 0.03173539930310159
Test statistic: 0.9736842105263158
BinomTestResult(k=37, n=38, alternative='greater', statistic=0.9736842105263158, pvalue=0.03173539930310159)


In [25]:
df.at[df.shape[0]+1, 'correct'] = np.mean(df['correct'])
df.at[df.shape[0], 'sarcastic'] = np.mean(df['sarcastic'])
df.at[df.shape[0]+1, 'correct'] = f"P-value: {round(result.pvalue, 3)}"

In [26]:
df

,ID,original_ID,tweet,sarcastic,rephrase,prediction,correct
0,5002.0,test_a_921,I really loved the drive into work today in th...,1.000000,NaN,1,1
1,1377.0,sign_3530,i just looooove getting big cases on friday af...,1.000000,i just hate getting big cases on friday aftern...,1,1
2,3297.0,sign_12376,i just love it when people near and dear to me...,1.000000,no one seems to care that i'm sick,1,1
3,2404.0,train_2773,man I really hate seeing two fucking idiots ra...,0.000000,NaN,0,1
4,10364.0,test_a_873,I really hate being stuck at home thanks to th...,0.000000,NaN,0,1
5,13608.0,train_2450,I absolutely hate when players do that dumb ea...,0.000000,NaN,0,1
6,4821.0,sign_3549,i absolutely love how my mom doesn't believe a...,1.000000,i really don't like how my mom doesn't believe...,1,1
7,8042.0,sign_12009,i really love enrollments,1.000000,i really hate enrollments,1,1
8,15087.0,sign_12389,i just love going down my timeline to see one ...,1.000000,i hate the negative posts on my timeline,1,1
9,15507.0,sign_2773,i just love having to get my car towed on a su...,1.000000,i just hate having to get my car towed on a su...,1,1


In [27]:
df.to_csv('exp3_results_27112024.csv', index=False)

# Rule-based Sarcasm Interpreter

In [ ]:
#Final sarcasm classification function for when we have established the best threshold
def interpret_sarcasm(tweet, verb, noun_phrase, threshold=0.528):
    tweet_sentiment = classify_sentiment(tweet)
    tweet_no_v_sentiment = classify_sentiment(tweet.replace(verb, ""))
    tweet_no_np_sentiment = classify_sentiment(tweet.replace(noun_phrase, ""))
    v_sentiment = tweet_sentiment - tweet_no_v_sentiment
    np_sentiment = tweet_sentiment - tweet_no_np_sentiment

    if np.linalg.norm(v_sentiment - np_sentiment)/6.3885164 > threshold:
      return negator.negate_sentence(tweet)
    else:
      return "The tweet is not sarcastic."

In [ ]:
#TEST!!!!!!!!!!!!!!!!!!!!!!!!!!
#Final sarcasm classification function for when we have established the best threshold
def interpret_sarcasm(tweet, verb, noun_phrase, threshold=0.528):
    tweet_sentiment = classify_sentiment(tweet)
    tweet_no_v_sentiment = classify_sentiment(tweet.replace(verb, ""))
    tweet_no_np_sentiment = classify_sentiment(tweet.replace(noun_phrase, ""))
    v_sentiment = tweet_sentiment - tweet_no_v_sentiment
    np_sentiment = tweet_sentiment - tweet_no_np_sentiment

    if np.linalg.norm(v_sentiment - np_sentiment)/6.3885164 > threshold:
      tweet_no_np = tweet.replace(noun_phrase, "")
      negated_part = negator.negate_sentence(tweet_no_np)
      return negated_part + " " + noun_phrase
    else:
      return "The tweet is not sarcastic."